### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME','ORGANIZATION'],1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1283        1
C1732        1
C2150        1
C4200        1
C1900        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_count[classification_count < 1800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Look at INCOME_AMT value counts for binning
income_count = application_df.INCOME_AMT.value_counts()
income_count

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Determine which values to replace if counts are less than ..?
replace_income = list(income_count[income_count < 500].index)

# Replace in dataframe
for cls in replace_income:
    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
Other              564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [10]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
INCOME_AMT                7
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_ProductDev,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values.reshape(-1,1)
X = application_df.drop(['IS_SUCCESSFUL','SPECIAL_CONSIDERATIONS_Y'],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=1,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3700      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 8,801
Trainable params: 8,801
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5918 - accuracy: 0.7079
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5623 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5597 - accuracy: 0.7277
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5603 - accuracy: 0.7303
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5571 - accuracy: 0.7325
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5595 - accuracy: 0.7301
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5596 - accuracy: 0.7290
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5625 - accuracy: 0.7243: 0s -
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5577 - accuracy: 0.7270
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5581 - 

Epoch 81/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7321
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7330
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7312
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7349
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7329
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accuracy: 0.7369
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7328
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7356
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7339
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5460

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5714 - accuracy: 0.7240
Loss: 0.5714176297187805, Accuracy: 0.7239649891853333


In [19]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization2/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization2/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
 69/804 [=>............................] - ETA: 1s - loss: 0.5486 - accuracy: 0.7446
Epoch 00001: saving model to checkpoints/Optimization2/weights.01.hdf5
172/804 [=====>........................] - ETA: 0s - loss: 0.5455 - accuracy: 0.7420
Epoch 00001: saving model to checkpoints/Optimization2/weights.01.hdf5
271/804 [=========>....................] - ETA: 0s - loss: 0.5447 - accuracy: 0.7417
Epoch 00001: saving model to checkpoints/Optimization2/weights.01.hdf5
392/804 [=============>................] - ETA: 0s - loss: 0.5449 - accuracy: 0.7408
Epoch 00001: saving model to checkpoints/Optimization2/weights.01.hdf5
482/804 [================>.............] - ETA: 0s - loss: 0.5455 - accuracy: 0.7400
Epoch 00001: saving model to checkpoints/Optimization2/weights.01.hdf5
589/804 [====================>.........] - ETA: 0s - loss: 0.5459 - accuracy: 0.7392
Epoch 00001: saving model to checkpoints/Optimization2/weights.01.hdf5
695/804 [========================>.....] - ETA: 0s -

 49/804 [>.............................] - ETA: 1s - loss: 0.5546 - accuracy: 0.7404
Epoch 00007: saving model to checkpoints/Optimization2/weights.07.hdf5
160/804 [====>.........................] - ETA: 1s - loss: 0.5503 - accuracy: 0.7406
Epoch 00007: saving model to checkpoints/Optimization2/weights.07.hdf5
266/804 [========>.....................] - ETA: 0s - loss: 0.5475 - accuracy: 0.7417
Epoch 00007: saving model to checkpoints/Optimization2/weights.07.hdf5
347/804 [===========>..................] - ETA: 0s - loss: 0.5456 - accuracy: 0.7423
Epoch 00007: saving model to checkpoints/Optimization2/weights.07.hdf5
458/804 [================>.............] - ETA: 0s - loss: 0.5451 - accuracy: 0.7415
Epoch 00007: saving model to checkpoints/Optimization2/weights.07.hdf5
569/804 [====================>.........] - ETA: 0s - loss: 0.5451 - accuracy: 0.7407
Epoch 00007: saving model to checkpoints/Optimization2/weights.07.hdf5
639/804 [======================>.......] - ETA: 0s - loss: 0.545

 28/804 [>.............................] - ETA: 1s - loss: 0.5483 - accuracy: 0.7219
Epoch 00013: saving model to checkpoints/Optimization2/weights.13.hdf5
128/804 [===>..........................] - ETA: 1s - loss: 0.5359 - accuracy: 0.7393
Epoch 00013: saving model to checkpoints/Optimization2/weights.13.hdf5
224/804 [=======>......................] - ETA: 0s - loss: 0.5345 - accuracy: 0.7434
Epoch 00013: saving model to checkpoints/Optimization2/weights.13.hdf5
331/804 [===========>..................] - ETA: 0s - loss: 0.5356 - accuracy: 0.7438
Epoch 00013: saving model to checkpoints/Optimization2/weights.13.hdf5
435/804 [===============>..............] - ETA: 0s - loss: 0.5376 - accuracy: 0.7425
Epoch 00013: saving model to checkpoints/Optimization2/weights.13.hdf5
533/804 [==================>...........] - ETA: 0s - loss: 0.5393 - accuracy: 0.7411
Epoch 00013: saving model to checkpoints/Optimization2/weights.13.hdf5
632/804 [======================>.......] - ETA: 0s - loss: 0.540

  1/804 [..............................] - ETA: 2s - loss: 0.4899 - accuracy: 0.8125
Epoch 00019: saving model to checkpoints/Optimization2/weights.19.hdf5
104/804 [==>...........................] - ETA: 1s - loss: 0.5417 - accuracy: 0.7430
Epoch 00019: saving model to checkpoints/Optimization2/weights.19.hdf5
205/804 [======>.......................] - ETA: 0s - loss: 0.5365 - accuracy: 0.7455
Epoch 00019: saving model to checkpoints/Optimization2/weights.19.hdf5
302/804 [==========>...................] - ETA: 0s - loss: 0.5369 - accuracy: 0.7444
Epoch 00019: saving model to checkpoints/Optimization2/weights.19.hdf5
411/804 [==============>...............] - ETA: 0s - loss: 0.5375 - accuracy: 0.7438
Epoch 00019: saving model to checkpoints/Optimization2/weights.19.hdf5
520/804 [==================>...........] - ETA: 0s - loss: 0.5388 - accuracy: 0.7425
Epoch 00019: saving model to checkpoints/Optimization2/weights.19.hdf5
588/804 [====================>.........] - ETA: 0s - loss: 0.539

 84/804 [==>...........................] - ETA: 1s - loss: 0.5607 - accuracy: 0.7239
Epoch 00025: saving model to checkpoints/Optimization2/weights.25.hdf5
189/804 [======>.......................] - ETA: 0s - loss: 0.5567 - accuracy: 0.7271
Epoch 00025: saving model to checkpoints/Optimization2/weights.25.hdf5
263/804 [========>.....................] - ETA: 0s - loss: 0.5538 - accuracy: 0.7299
Epoch 00025: saving model to checkpoints/Optimization2/weights.25.hdf5
382/804 [=============>................] - ETA: 0s - loss: 0.5521 - accuracy: 0.7318
Epoch 00025: saving model to checkpoints/Optimization2/weights.25.hdf5
474/804 [================>.............] - ETA: 0s - loss: 0.5511 - accuracy: 0.7328
Epoch 00025: saving model to checkpoints/Optimization2/weights.25.hdf5
588/804 [====================>.........] - ETA: 0s - loss: 0.5502 - accuracy: 0.7337
Epoch 00025: saving model to checkpoints/Optimization2/weights.25.hdf5
701/804 [=========================>....] - ETA: 0s - loss: 0.549

...

393/804 [=============>................] - ETA: 0s - loss: 0.5417 - accuracy: 0.7368
Epoch 00073: saving model to checkpoints/Optimization2/weights.73.hdf5
486/804 [=================>............] - ETA: 0s - loss: 0.5452 - accuracy: 0.7358
Epoch 00073: saving model to checkpoints/Optimization2/weights.73.hdf5
583/804 [====================>.........] - ETA: 0s - loss: 0.5470 - accuracy: 0.7356
Epoch 00073: saving model to checkpoints/Optimization2/weights.73.hdf5
680/804 [========================>.....] - ETA: 0s - loss: 0.5482 - accuracy: 0.7353
Epoch 00073: saving model to checkpoints/Optimization2/weights.73.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5491 - accuracy: 0.7351
Epoch 74/100
  1/804 [..............................] - ETA: 1s - loss: 0.6148 - accuracy: 0.6562
Epoch 00074: saving model to checkpoints/Optimization2/weights.74.hdf5
 83/804 [==>...........................] - ETA: 1s - loss: 0.5410 - accuracy: 0.7406
Epoch 00074: saving model to chec

468/804 [================>.............] - ETA: 0s - loss: 0.5491 - accuracy: 0.7342
Epoch 00079: saving model to checkpoints/Optimization2/weights.79.hdf5
572/804 [====================>.........] - ETA: 0s - loss: 0.5483 - accuracy: 0.7343
Epoch 00079: saving model to checkpoints/Optimization2/weights.79.hdf5
675/804 [========================>.....] - ETA: 0s - loss: 0.5479 - accuracy: 0.7343
Epoch 00079: saving model to checkpoints/Optimization2/weights.79.hdf5
757/804 [===========================>..] - ETA: 0s - loss: 0.5478 - accuracy: 0.7343
Epoch 00079: saving model to checkpoints/Optimization2/weights.79.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7343
Epoch 80/100
 69/804 [=>............................] - ETA: 1s - loss: 0.5402 - accuracy: 0.7369
Epoch 00080: saving model to checkpoints/Optimization2/weights.80.hdf5
180/804 [=====>........................] - ETA: 1s - loss: 0.5443 - accuracy: 0.7345
Epoch 00080: saving model to chec

538/804 [===================>..........] - ETA: 0s - loss: 0.5447 - accuracy: 0.7350
Epoch 00085: saving model to checkpoints/Optimization2/weights.85.hdf5
636/804 [======================>.......] - ETA: 0s - loss: 0.5454 - accuracy: 0.7347
Epoch 00085: saving model to checkpoints/Optimization2/weights.85.hdf5
748/804 [==========================>...] - ETA: 0s - loss: 0.5458 - accuracy: 0.7346
Epoch 00085: saving model to checkpoints/Optimization2/weights.85.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5459 - accuracy: 0.7346
Epoch 86/100
 31/804 [>.............................] - ETA: 1s - loss: 0.5377 - accuracy: 0.7341
Epoch 00086: saving model to checkpoints/Optimization2/weights.86.hdf5
155/804 [====>.........................] - ETA: 2s - loss: 0.5372 - accuracy: 0.7396
Epoch 00086: saving model to checkpoints/Optimization2/weights.86.hdf5
255/804 [========>.....................] - ETA: 2s - loss: 0.5388 - accuracy: 0.7388
Epoch 00086: saving model to chec

613/804 [=====================>........] - ETA: 0s - loss: 0.5454 - accuracy: 0.7353
Epoch 00091: saving model to checkpoints/Optimization2/weights.91.hdf5
721/804 [=========================>....] - ETA: 0s - loss: 0.5458 - accuracy: 0.7349
Epoch 00091: saving model to checkpoints/Optimization2/weights.91.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accuracy: 0.7348
Epoch 92/100
 27/804 [>.............................] - ETA: 1s - loss: 0.5379 - accuracy: 0.7372
Epoch 00092: saving model to checkpoints/Optimization2/weights.92.hdf5
117/804 [===>..........................] - ETA: 1s - loss: 0.5453 - accuracy: 0.7355
Epoch 00092: saving model to checkpoints/Optimization2/weights.92.hdf5
223/804 [=======>......................] - ETA: 0s - loss: 0.5460 - accuracy: 0.7345
Epoch 00092: saving model to checkpoints/Optimization2/weights.92.hdf5
297/804 [==========>...................] - ETA: 0s - loss: 0.5463 - accuracy: 0.7338
Epoch 00092: saving model to chec

609/804 [=====================>........] - ETA: 0s - loss: 0.5460 - accuracy: 0.7350
Epoch 00097: saving model to checkpoints/Optimization2/weights.97.hdf5
707/804 [=========================>....] - ETA: 0s - loss: 0.5461 - accuracy: 0.7349
Epoch 00097: saving model to checkpoints/Optimization2/weights.97.hdf5
804/804 [==============================] - 3s 4ms/step - loss: 0.5462 - accuracy: 0.7348
Epoch 98/100
  1/804 [..............................] - ETA: 3s - loss: 0.5583 - accuracy: 0.7500
Epoch 00098: saving model to checkpoints/Optimization2/weights.98.hdf5
102/804 [==>...........................] - ETA: 2s - loss: 0.5452 - accuracy: 0.7308
Epoch 00098: saving model to checkpoints/Optimization2/weights.98.hdf5
196/804 [======>.......................] - ETA: 1s - loss: 0.5457 - accuracy: 0.7320
Epoch 00098: saving model to checkpoints/Optimization2/weights.98.hdf5
311/804 [==========>...................] - ETA: 1s - loss: 0.5461 - accuracy: 0.7326
Epoch 00098: saving model to chec

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5910 - accuracy: 0.7224
Loss: 0.5909780859947205, Accuracy: 0.722449004650116


In [21]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization2.h5')